Этап 1. Детекция и трекинг машин

Используйте YOLOv8 (Ultralytics) для детекции автомобилей (класс car в COCO).
Активируйте встроенный трекинг (например, BoT-SORT) для сохранения уникальных ID объектов между кадрами.
Отображайте bounding boxes с ID и классом объектов.

Этап 2. Определение линии подсчёта

Задайте две точки ((x1, y1) и (x2, y2)), определяющие линию пересечения (например, горизонтальную или вертикальную).
Линия должна отображаться на кадре.

Этап 3. Логика подсчета пересечений

Для каждого трека определите его центр (центр bounding box).
Проверяйте, пересекает ли траектория движения объекта заданную линию.
Учитывайте пересечение только в одном направлении (например, слева направо).
Исключите повторный подсчет для одного и того же объекта (после пересечения он не должен учитываться снова).

Этап 4. Визуализация

Отображайте bounding boxes, ID объектов, текущее положение центра.
Выводите линию подсчёта и общее количество пересечений в углу кадра.
Для пересекших объектов можно менять цвет bounding box или добавлять метку.

Этап 5. Тестирование

Протестируйте решение на видео с движущимися автомобилями (например, пример видео).
Дополнительные задания (для повышения итогового балла):


В качестве дополнительного задания я определила построение бокса на выделенной полосе, чтобы считать машины, которые на эту полосу заезжают

In [ ]:
!pip install lap ultralytics opencv-python

In [1]:
import warnings
warnings.filterwarnings("ignore")


In [2]:
import torch
import torchvision
print(torch.__version__)
print(torchvision.__version__)

2.5.1
0.20.1


In [3]:
from ultralytics import YOLO
import cv2
import time

Этапы работы были следущими

Видео (уже обрезанное под нужный фрагмент) обработано для уменьшения FPS и разрешения

In [4]:
cap = cv2.VideoCapture("video/video_cropped.mp4")
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

target_fps = 10
skip = max(1, round(fps / target_fps))
size = (w//2, h//2)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('video/video_10fps.mp4', fourcc, target_fps, size)

frame_id = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_id += 1
    if frame_id % skip:
        continue
    small = cv2.resize(frame, size)
    out.write(small)

cap.release()
out.release()

Здесь реализованы все этапы: общий счетчик машин (однонаправленный), а также бокс под "запретную зону", который перекрывает выделенную полосу, и считает нарушителей

In [5]:
import cv2
import time
import math
import numpy as np
from ultralytics import YOLO



In [6]:
model = YOLO("yolov8m.pt")
model.tracker = "botsort.yaml"
classes = (2,)  # класс "машина"

cap = cv2.VideoCapture("video/video_10fps.mp4")

fps = cap.get(cv2.CAP_PROP_FPS)
frame_time = 1.0 / fps
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter("video/final_video.mp4", fourcc, fps, (w, h))



In [7]:
line_start = (0, h // 2) #тут рисуем линию и бокс, бокс наклонила на 15 градусов для подгонки под полосу на видео
line_end = (w, h // 2)
line_y = h // 2

box_width = w // 2
box_height = w // 16
box_angle_deg = -15
box_angle_rad = math.radians(box_angle_deg)

box_cx = w // 10
box_cy = h - 50

dx = box_width // 2
dy = box_height // 2



In [8]:
rect_pts = [] #соберем координаты "запретного" прямоугольника с учетом наклона
for sign_x, sign_y in [(-1, -1), (1, -1), (1, 1), (-1, 1)]:
    x = box_cx + sign_x * dx * math.cos(box_angle_rad) - sign_y * dy * math.sin(box_angle_rad)
    y = box_cy + sign_x * dx * math.sin(box_angle_rad) + sign_y * dy * math.cos(box_angle_rad)
    rect_pts.append([int(x), int(y)])
rect_pts_np = np.array(rect_pts, dtype=np.int32)



In [9]:
prev_centers = {} # сделаем счетчик как множество, чтобы не считать объекты дважды
counted_ids_line = set()
counted_ids_zone = set()
line_count = 0
zone_count = 0




In [10]:
while True: # читаем видео
    start = time.time()
    ret, frame = cap.read()
    if not ret:
        break

    results = model.track(frame, persist=True, classes=classes, tracker=model.tracker) # здесь все наши предсказания - рамки, id, координаты объектов, 
                                                                #которые мы ниже извлекаем
    frame = results[0].orig_img.copy()
    boxes = results[0].boxes
    annotated = frame.copy()

    if boxes is not None: # извлечем данные на каждую машину
        for box in boxes:
            if box.id is None:
                continue

            id = int(box.id.item())
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cx = (x1 + x2) // 2
            cy = (y1 + y2) // 2

            prev_center = prev_centers.get(id, (cx, cy))
            prev_cy = prev_center[1] # извлекаем координаты на предыдущий кадр, чтобы далее сравнить с последующими координатами (y)

            if id not in counted_ids_line and prev_cy > line_y and cy <= line_y: # тут сравниваем, выше ли каждый ранее детектированный
                                                                                #и зафиксированный на нужной полосе объект,
                                                                                #линии пересечения, если да, увеличиваем счетчик
                line_count += 1
                counted_ids_line.add(id)

            if id not in counted_ids_zone: #аналогично для выделенки
                if cv2.pointPolygonTest(rect_pts_np, (cx, cy), False) >= 0:
                    zone_count += 1
                    counted_ids_zone.add(id)


            prev_centers[id] = (cx, cy)

            if id in counted_ids_zone: # выделим заехавшего на выделенку красным цветом, перескающих горизонтальную линию - зеленым
                color = (0, 0, 255)
            elif id in counted_ids_line:
                color = (0, 255, 0)
            else:
                color = (255, 0, 0)

            cv2.rectangle(annotated, (x1, y1), (x2, y2), color, 2) #пропишем визуализацию остальных объктов
            cv2.circle(annotated, (cx, cy), 4, color, -1)
            cv2.putText(annotated, f"ID: {id}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)


    cv2.line(annotated, line_start, line_end, (0, 255, 255), 2)
    cv2.putText(annotated, f"Line crossed: {line_count}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

    cv2.polylines(annotated, [rect_pts_np], isClosed=True, color=(255, 255, 0), thickness=2)
    cv2.putText(annotated, f"Restricted zone crossed: {zone_count}", (10, 70),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

    # видео
    out.write(annotated)

    elapsed = time.time() - start
    to_wait = frame_time - elapsed
    if to_wait > 0:
        time.sleep(to_wait)

    if cv2.waitKey(1) & 0xFF == 27:
        break



0: 384x640 7 cars, 36.8ms
Speed: 1.7ms preprocess, 36.8ms inference, 88.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 16.1ms
Speed: 1.0ms preprocess, 16.1ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 16.2ms
Speed: 2.6ms preprocess, 16.2ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 16.3ms
Speed: 2.5ms preprocess, 16.3ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 16.3ms
Speed: 2.5ms preprocess, 16.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 16.3ms
Speed: 2.6ms preprocess, 16.3ms inference, 3.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 16.3ms
Speed: 1.9ms preprocess, 16.3ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 16.1ms
Speed: 1.0ms preprocess, 16.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 3

In [11]:
cap.release()
out.release()

Заключение: все 5 этапов и 1 дополнительный реализованы, оба счетчика отрабатывают правильно, проблем с детекцией нет, визуализация проработана